In [1]:
%pip install --upgrade --quiet langchain

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install streamlit 


Defaulting to user installation because normal site-packages is not writeable
  Using cached streamlit-1.38.0-py2.py3-none-any.whl (8.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 KB 2.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 10.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.1/658.1 KB 16.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
import getpass
import os
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def divide(first_int: int, second_int: int) -> int:
    """Divide two integers together."""
    return first_int / second_int

In [ ]:
@tool
def divide(first_int: int, second_int: int) -> int:
    """Divide two integers together."""
    return first_int / second_int

In [3]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [13]:
divide.invoke({"first_int": 4, "second_int": 5})



0.8

In [5]:
%pip install -qU langchain-openai



Note: you may need to restart the kernel to use updated packages.


In [6]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [14]:
llm_with_tools = llm.bind_tools([multiply, divide])



In [19]:
msg = llm_with_tools.invoke("whats 5 multiplies by  two, then divide the results by three")
msg.tool_calls

[{'name': 'multiply',
  'args': {'first_int': 5, 'second_int': 2},
  'id': 'call_P6izhMFxd0SsSNn0IA8PV0Gy',
  'type': 'tool_call'},
 {'name': 'divide',
  'args': {'first_int': 10, 'second_int': 3},
  'id': 'call_tARpiNBoL7m2fjSHywpD7hod',
  'type': 'tool_call'}]

In [17]:
msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_V1y5wHbJhUHfYZP3gNbvLLx6', 'function': {'arguments': '{"first_int":5,"second_int":2}', 'name': 'divide'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 81, 'total_tokens': 100, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-835b37a9-1325-456d-910f-c44831eab452-0', tool_calls=[{'name': 'divide', 'args': {'first_int': 5, 'second_int': 2}, 'id': 'call_V1y5wHbJhUHfYZP3gNbvLLx6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 81, 'output_tokens': 19, 'total_tokens': 100})

In [23]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

In [24]:
from operator import itemgetter
from typing import Dict, List, Union

from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

tools = [multiply, exponentiate, add]
llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}


def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


chain = llm_with_tools | call_tools

In [26]:
chain.invoke("What is 2+2 then multiply the result by 3")



[{'name': 'add',
  'args': {'first_int': 2, 'second_int': 2},
  'id': 'call_DSmckdTZYgjeGFkpxx7EI3kQ',
  'type': 'tool_call',
  'output': 4},
 {'name': 'multiply',
  'args': {'first_int': 4, 'second_int': 3},
  'id': 'call_Pm3owtuGcDRkQb3ac0ZBSYdp',
  'type': 'tool_call',
  'output': 12}]